# 装饰器

装饰器的作用就是为已经存在的对象添加额外的功能

装饰器本质上是一个Python函数，返回值也是一个函数对象

它经常用于有切面需求的场景，比如：插入日志、性能测试、事务处理、缓存、权限校验等场景。装饰器是解决这类问题的绝佳设计，有了装饰器，我们就可以抽离出大量与函数功能本身无关的雷同代码并继续重用。

装饰器在Python使用如此方便都要归因于Python的函数能够像普通的对象一样能作为参数传递给其他函数，可以被复制给其他变量，可以作为返回值，可以被定义在另一个函数内

@符号是Python装饰器的语法糖，它放在一个函数开始定义的地方，像一顶帽子一样戴在这个函数的头上。和这个函数绑定在一起。在我们调用这个函数的时候，第一件事并不是执行这个函数，而是将这个函数做为参数传入它头顶上这顶帽子，这顶帽子我们称之为**装饰函数** 或 **装饰器**。

## 01 普通装饰器


In [7]:
import logging
def use_logging(func):
    def wrapper(*args, **kwargs):
        logging.warn("%s is running" % func.__name__)
        return func(*args, **kwargs)
    return wrapper

@use_logging
def foo():
    print("i am foo")

@use_logging
def bar():
    print("i am bar")

bar()

/home/zjw/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
  after removing the cwd from sys.path.


i am bar


In [8]:
# 时间计数器
import time
def timer(func):
    def wrapper(*args, **kwargs):
        t1 = time.time()
        func(*args, **kwargs)
        t2 = time.time()
        print('cost time:{} seconds'.format(t2-t1))
    return wrapper

@timer
def want_sleep(sleep_time):
    time.sleep(sleep_time)
    
want_sleep(5)

cost time:5.0033745765686035 seconds


## 02 带参数装饰器

在上面的装饰器调用中，比如@use_logging，该装饰器唯一的参数就是执行业务的函数。装饰器的语法允许我们在调用时，提供其它参数，比如@decorator(a)。

传参需要两层嵌套

In [4]:
def use_logging(level):
    def decorator(func):
        def wrapper(*args, **kwargs):
            if level == "warn":
                logging.warn("%s is running" % func.__name__)
            return func(*args)
        return wrapper
    return decorator

@use_logging(level="warn")
def foo(name='foo'):
    print("i am %s" % name)

foo()

/home/zjw/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
  """


i am foo


## 03 不带参数的类装饰器

以上都是基于函数实现的装饰器，在阅读别人代码时，还可以时常发现还有基于类实现的装饰器。

基于类装饰器的实现，必须实现 \_\_call\_\_ 和 \_\_init\_\_两个内置函数。 \_\_init\_\_ ：接收被装饰函数 \_\_call\_\_ ：实现装饰逻辑。当使用 @ 形式将装饰器附加到函数上时，就会调用此方法。

In [9]:
class logger(object):
    def __init__(self, func):
        self.func = func

    def __call__(self, *args, **kwargs):
        print("[INFO]: the function {func}() is running..."\
            .format(func=self.func.__name__))
        return self.func(*args, **kwargs)

@logger
def say(something):
    print("say {}!".format(something))

say("hello")

[INFO]: the function say() is running...
say hello!


## 04 带参数的类装饰器

上面不带参数的例子，你发现没有，只能打印`INFO`级别的日志，正常情况下，我们还需要打印`DEBUG `,`WARNING`等级别的日志。 这就需要给类装饰器传入参数，给这个函数指定级别了。

带参数和不带参数的类装饰器有很大的不同。

\_\_init\_\_ ：不再接收被装饰函数，而是接收传入参数。 \_\_call\_\_ ：接收被装饰函数，实现装饰逻辑。


In [10]:
class logger(object):
    def __init__(self, level='INFO'):
        self.level = level

    def __call__(self, func): # 接受函数
        def wrapper(*args, **kwargs):
            print("[{level}]: the function {func}() is running..."\
                .format(level=self.level, func=func.__name__))
            func(*args, **kwargs)
        return wrapper  #返回函数

@logger(level='WARNING')
def say(something):
    print("say {}!".format(something))

say("hello")

[WARNING]: the function say() is running...
say hello!


## 05 wraps装饰器



In [11]:
def wrapper(func):
    def inner_function():
        pass
    return inner_function

@wrapper
def wrapped():
    pass

print(wrapped.__name__)

inner_function


为什么返回的不是`func`？不难看出，因为上边执行 `func` 与下面  `decorator(func)` 是等价的， `func.__name__` 是等价于下面`decorator(func).__name_`_ 的，那当然名字是 `inner_function`

In [12]:
def wrapper(func):
    def inner_function():
        pass
    return inner_function

def wrapped():
    pass

print(wrapper(wrapped).__name__)

inner_function


那如何避免这种情况的产生？方法是使用 `functools.wraps` 装饰器，它的作用就是将 **被修饰的函数(wrapped)** 的一些属性值赋值给 修饰器函数**(wrapper)** ，最终让属性的显示更符合我们的直觉。

In [13]:
from functools import wraps

def wrapper(func):
    @wraps(func)
    def inner_function():
        pass
    return inner_function

@wrapper
def wrapped():
    pass

print(wrapped.__name__)

wrapped


## 内置装饰器：property

以上，我们介绍的都是自定义的装饰器。

其实Python语言本身也有一些装饰器。比如property这个内建装饰器，我们再熟悉不过了。

它通常存在于类中，可以将一个函数定义成一个属性，属性的值就是该函数return的内容。

通常我们给实例绑定属性是这样的

In [14]:
class Student():
    def __init__(self, name, age=None):
        self.name = name
        self.age = age
    
# 实例化
XiaoMing = Student("小明")

# 添加属性
XiaoMing.age=25

# 查询属性
XiaoMing.age

# 删除属性
del XiaoMing.age

但是稍有经验的开发人员，一下就可以看出，**这样直接把属性暴露出去，虽然写起来很简单，但是并不能对属性的值做合法性限制**。为了实现这个功能，我们可以这样写。

In [17]:
class Student(object):
    def __init__(self, name):
        self.name = name
        self.name = None

    def set_age(self, age):
        if not isinstance(age, int):
            raise ValueError('输入不合法：年龄必须为数值!')
        if not 0 < age < 100:
            raise ValueError('输入不合法：年龄范围必须0-100')
        self._age=age

    def get_age(self):
        return self._age

    def del_age(self):
        self._age = None


XiaoMing = Student("小明")

# 添加属性
XiaoMing.set_age(25)

# 查询属性
XiaoMing.get_age()

# 删除属性
XiaoMing.del_age()

上面的代码设计虽然可以变量的定义，但是可以发现不管是获取还是赋值（通过函数）都和我们平时见到的不一样。 按照我们思维习惯应该是这样的。

```python
# 赋值
XiaoMing.age = 25

# 获取
XiaoMing.age
```
那么这样的方式我们如何实现呢。请看下面的代码。


In [21]:
class Student(object):
    def __init__(self, name):
        self.name = name
        self.name = None

    @property
    def age(self):
        return self._age

    @age.setter
    def age(self, value):
        if not isinstance(value, int):
            raise ValueError('输入不合法：年龄必须为数值!')
        if not 0 < value < 100:
            raise ValueError('输入不合法：年龄范围必须0-100')
        self._age=value

    @age.deleter
    def age(self):
        del self._age

XiaoMing = Student("小明")

# 设置属性
XiaoMing.age = 25

# 查询属性
XiaoMing.age

# 删除属性
del XiaoMing.age

用`@property`装饰过的函数，会将一个函数定义成一个属性，属性的值就是该函数return的内容。同时，会将这个函数变成另外一个装饰器。就像后面我们使用的`@age.setter`和`@age.deleter`。`@age.setter` 使得我们可以使用`XiaoMing.age = 25`这样的方式直接赋值。 `@age.deleter` 使得我们可以使用`del XiaoMing.age`这样的方式来删除属性。